In [2]:
import pandas as pd
import os
import os.path as op
from recruitment_for_vs import *
import glob
import shutil
from phage_count_table import phage_contig_table
from recruitment_for_vs import summarize_by_contig
from summary import virus_class

In [3]:
wd = "/mnt/scgc/simon/simonsproject/bats248_vs"

to_start =  '''AG−891−A17
AG−892−P18
AG−893−J23
AG−894−C07
AG−895−P08
AG−897−A15  
AG−903−F19
AG−903−I06
AG−904−O13
AG−907−C19
AG−907−I10
AG−908−F15
AG-909-A05
AG−910−E05
AG−912−O18
AG−913−C05
AG−913−C17'''.replace("−","-").split()

## creating objects needed for munging mica results:

In [4]:
# cluster map created in 180125_orf_prodigal_setup_reclustering_and_qc.ipynb

out_map = "/mnt/scgc/simon/simonsproject/bats248_vs/clustering/cluster_map.txt"

def write_cluster_map(cmap, out_map):
    count = 0
    with open(out_map, "w") as oh:
        for k in cluster_map:
            print(k, k, sep = "\t", file=oh)
            for val in cluster_map[k]:
                print(val, k, sep="\t", file=oh)
                
def read_cluster_map(out_map):
    cmap = {}
    with open(out_map) as ih:
        for l in ih:
            vec = l.strip().split("\t")
            cmap[vec[0]] = vec[1]
    return cmap

In [5]:
phage_hits = pd.concat([pd.read_csv(i) for i in glob.glob(op.join(wd, "phage_hits*"))])

In [6]:
cmap = read_cluster_map(out_map)

In [14]:
sag = to_start[0]

In [15]:
prot_fasta = op.join(wd, 'prodigal', '{}_proteins.fasta'.format(sag))

In [18]:
phage_df = phage_contig_table(cmap, prot_fasta, phage_hits, outfile=None)

In [25]:
phage_df[:5]

,p1,p2,total_orfs,viral_phage_gene_fraction,viral2_phage_gene_fraction
contig,,,,,
AG-891-A17_NODE_1,3.0,0.0,166,0.018072,0.0
AG-891-A17_NODE_10,0.0,0.0,35,0.000000,0.0
AG-891-A17_NODE_11,0.0,0.0,21,0.000000,0.0
AG-891-A17_NODE_12,0.0,0.0,23,0.000000,0.0
AG-891-A17_NODE_13,0.0,0.0,30,0.000000,0.0


In [22]:
recruit_csv = op.join(wd, 'diamond','pergenome','{}_proteins_mg_diamond_recruitment_tbl.csv'.format(sag))

In [23]:
recruit_df = pd.read_csv(recruit_csv)

In [26]:
recruit_df[:5]

,contig,hit_mg-bac,hit_mg-vir,reads_mg-vir,reads_mg-bac,contig_length,fr_mg-bac,fr_mg-vir,ratio_virus_bacteria
0,AG-891-A17_NODE_1,44265.0,4208.0,5922080.0,8279226.0,147339.0,0.036287,0.004823,0.132902
1,AG-891-A17_NODE_10,5766.0,669.0,5922080.0,8279226.0,32598.0,0.021365,0.003465,0.162206
2,AG-891-A17_NODE_11,12122.0,993.0,5922080.0,8279226.0,24711.0,0.059251,0.006786,0.114522
3,AG-891-A17_NODE_12,8555.0,487.0,5922080.0,8279226.0,22815.0,0.045291,0.003604,0.079584
4,AG-891-A17_NODE_13,5673.0,635.0,5922080.0,8279226.0,22000.0,0.031146,0.004874,0.156486


In [27]:
merged = recruit_df.merge(phage_df.reset_index(), on='contig', how='outer')

In [29]:
result = merged.merge(virus_class(merged), on='contig', how='outer')

In [31]:
result[:10]

,contig,hit_mg-bac,hit_mg-vir,reads_mg-vir,reads_mg-bac,contig_length,fr_mg-bac,fr_mg-vir,ratio_virus_bacteria,p1,p2,total_orfs,viral_phage_gene_fraction,viral2_phage_gene_fraction,virus_class,virus_prob
0,AG-891-A17_NODE_1,44265.0,4208.0,5922080.0,8279226.0,147339.0,0.036287,0.004823,0.132902,3.0,0.0,166,0.018072,0.0,0.0,1.0
1,AG-891-A17_NODE_10,5766.0,669.0,5922080.0,8279226.0,32598.0,0.021365,0.003465,0.162206,0.0,0.0,35,0.000000,0.0,0.0,1.0
2,AG-891-A17_NODE_11,12122.0,993.0,5922080.0,8279226.0,24711.0,0.059251,0.006786,0.114522,0.0,0.0,21,0.000000,0.0,0.0,1.0
3,AG-891-A17_NODE_12,8555.0,487.0,5922080.0,8279226.0,22815.0,0.045291,0.003604,0.079584,0.0,0.0,23,0.000000,0.0,0.0,1.0
4,AG-891-A17_NODE_13,5673.0,635.0,5922080.0,8279226.0,22000.0,0.031146,0.004874,0.156486,0.0,0.0,30,0.000000,0.0,0.0,1.0
5,AG-891-A17_NODE_14,4447.0,384.0,5922080.0,8279226.0,20404.0,0.026325,0.003178,0.120720,0.0,0.0,22,0.000000,0.0,0.0,1.0
6,AG-891-A17_NODE_15,8548.0,587.0,5922080.0,8279226.0,19050.0,0.054198,0.005203,0.096004,0.0,0.0,14,0.000000,0.0,0.0,1.0
7,AG-891-A17_NODE_16,6930.0,430.0,5922080.0,8279226.0,17004.0,0.049226,0.004270,0.086746,0.0,0.0,15,0.000000,0.0,0.0,1.0
8,AG-891-A17_NODE_17,422.0,89.0,5922080.0,8279226.0,13103.0,0.003890,0.001147,0.294844,0.0,0.0,13,0.000000,0.0,0.0,1.0
9,AG-891-A17_NODE_18,3190.0,163.0,5922080.0,8279226.0,12781.0,0.030146,0.002154,0.071435,0.0,0.0,15,0.000000,0.0,0.0,1.0


In [32]:
try:
    os.mkdir("/mnt/scgc/simon/simonsproject/bats248_vs/summary/")
except:
    print("oh well")

In [8]:
allsags = [op.basename(i).split("_")[0] for i in glob.glob("/mnt/scgc/simon/simonsproject/bats248_vs/prodigal/*_proteins.fasta")]
allsags[0]

'AG-359-B02'

In [10]:
not_found = []
merge_error = []
summaries = 0

for sag in allsags:
    prot_fasta = op.join(wd, 'prodigal', '{}_proteins.fasta'.format(sag))
    recruit_csv = op.join(wd, 'diamond','pergenome','{}_proteins_mg_diamond_recruitment_tbl.csv'.format(sag))
    out_csv = op.join(wd, 'summary', '{}_summary.csv'.format(sag))
    
    if op.exists(out_csv):
        summaries += 1
        continue
    try:
        for f in [prot_fasta, recruit_csv]:
            try:
                assert op.exists(f)
            except:
                print("{} not found".format(f))
                not_found.append(f)
                raise OSError("file not found")
    except:
        continue
        
    phage_df = phage_contig_table(cmap, prot_fasta, phage_hits, outfile=None)
    recruit_df = pd.read_csv(recruit_csv)

    merged = recruit_df.merge(phage_df.reset_index(), on='contig', how='outer')
    try:
        result = merged.merge(virus_class(merged), on='contig', how='outer')
    except:
        merge_error.append(sag)
        continue
    result.to_csv(out_csv, index=False)
    summaries += 1

/mnt/scgc/simon/simonsproject/bats248_vs/diamond/pergenome/AG-918-A17_proteins_mg_diamond_recruitment_tbl.csv not found
/mnt/scgc/simon/simonsproject/bats248_vs/diamond/pergenome/AG-918-P23_proteins_mg_diamond_recruitment_tbl.csv not found
/mnt/scgc/simon/simonsproject/bats248_vs/diamond/pergenome/AG-894-J09_proteins_mg_diamond_recruitment_tbl.csv not found


In [10]:
missing_sags = [op.basename(i).split("_")[0] for i in not_found]

In [11]:
recruitment_required = []
summary_generation_required = []

for s in missing_sags:
    recruit_files = glob.glob("/mnt/scgc/simon/simonsproject/bats248_vs/diamond/pergenome/{s}*".format(s=s))
    print(s, len(recruit_files), sep="\t")
    if len(recruit_files) == 0:
        recruitment_required.append(s)
    elif len(recruit_files) == 5:
        summary_generation_required.append(s)

AG-904-P07	5
AG-905-J06	5
AG-909-J16	5
AG-910-J09	5
AG-910-P20	5
AG-917-P16	5
AG-918-A17	5
AG-918-P23	5
AG-894-J09	5
AG-897-O20	5
AG-361-A09	5
AG-361-A19	5
AG-361-A20	5
AG-361-B11	5
AG-361-B18	5
AG-361-C03	5
AG-361-C05	5
AG-361-C13	5
AG-361-C16	5
AG-361-C19	5
AG-361-D10	5
AG-361-D11	5
AG-361-D21	5
AG-361-E07	5
AG-361-E13	5
AG-361-E16	5
AG-361-E22	5
AG-361-F03	5
AG-361-F05	5
AG-361-F07	5
AG-361-F19	5
AG-361-G08	5
AG-361-G09	5
AG-361-G16	5
AG-361-G19	5
AG-361-I03	5
AG-361-I04	5
AG-361-I08	5
AG-361-I22	5
AG-361-J02	5
AG-361-J07	5
AG-361-J09	5
AG-361-J10	5
AG-361-J14	5
AG-361-J17	5
AG-361-J22	5
AG-361-K16	5
AG-361-L02	5
AG-361-L07	5
AG-361-L13	5
AG-361-L15	5
AG-361-L16	5
AG-361-M04	5
AG-361-M09	5
AG-361-M10	5
AG-361-M13	5
AG-361-M14	5
AG-361-M19	5
AG-361-M20	5
AG-361-N03	5
AG-361-N04	5
AG-361-N11	5
AG-361-N17	5
AG-361-N21	5
AG-361-O03	5
AG-361-O10	5
AG-361-O16	5
AG-361-O17	5
AG-361-O18	5
AG-361-O19	5
AG-361-P10	5
AG-361-P15	5
AG-362-A02	5
AG-362-A03	5
AG-362-A08	5
AG-362-A16	5
AG-362-A20	5

In [12]:
" ".join(recruitment_required)

''

In [13]:
len(recruitment_required)

0

## all SAGs where recruitment was required were for the small particle sorts; they are all now batch submitted for recruitment

...recruitment has finished.  final table generation appears to have failed for all of them.

In [16]:
sag = summary_generation_required[0]
#sag = allsags[0]

In [17]:
sag

'AG-904-P07'

In [18]:
bac_tsv = "/mnt/scgc/simon/simonsproject/bats248_vs/diamond/pergenome/{}_proteins_vs_LineP-all.tsv.gz".format(sag)
vir_tsv = "/mnt/scgc/simon/simonsproject/bats248_vs/diamond/pergenome/{}_proteins_vs_POV.tsv.gz".format(sag)

In [19]:
plate = "-".join(sag.split("-")[:2])
contig_file = "/mnt/scgc/simon/simonsproject/bats248_contigs/coassemblies/{plate}/{sag}_contigs.fasta".format(plate=plate, sag=sag)
assert op.exists(contig_file), "could not find {}".format(contig_file)

In [20]:
from recruitment_for_vs import construct_recruit_tbl

read_count_dict = {'vir_reads':5922080, 'bac_reads':8279226}

In [21]:
tbl = construct_recruit_tbl(vir_tsv, bac_tsv, read_count_dict, contig_file)

doesn't look like input contig file is in gff format.  Will assume that contig name is embedded in the ORF name.
looks like input config fiel is in fasta format.


In [22]:
tbl

,contig,hit_mg-bac,hit_mg-vir,reads_mg-vir,reads_mg-bac,contig_length,fr_mg-bac,fr_mg-vir,ratio_virus_bacteria
0,AG-904-P07_NODE_3,NaN,1,5922080.0,8279226.0,4517.0,NaN,0.000037,NaN


In [30]:
read_count_dict = {'vir_reads':5922080, 'bac_reads':8279226}

missing = []
for sag in summary_generation_required:
    bac_tsv = "/mnt/scgc/simon/simonsproject/bats248_vs/diamond/pergenome/{}_proteins_vs_LineP-all.tsv.gz".format(sag)
    vir_tsv = "/mnt/scgc/simon/simonsproject/bats248_vs/diamond/pergenome/{}_proteins_vs_POV.tsv.gz".format(sag)
    plate = "-".join(sag.split("-")[:2])
    contig_file = "/mnt/scgc/simon/simonsproject/bats248_contigs/extracellular/{sag}_contigs.fasta".format(plate=plate, sag=sag)
    out_tbl = "/mnt/scgc/simon/simonsproject/bats248_vs/diamond/pergenome/{sag}__proteins_mg_diamond_recruitment_tbl.csv".format(sag=sag)
    if op.exists(out_tbl):
        continue
        
    exists=True
    for f in [bac_tsv, vir_tsv, contig_file]:

        if not op.exists(f):
            exists = False
            missing.append(sag)
            continue
       
    if exists == False:
        continue
            
    tbl = construct_recruit_tbl(vir_tsv, bac_tsv, read_count_dict, contig_file)      
    tbl.to_csv(out_tbl)

doesn't look like input contig file is in gff format.  Will assume that contig name is embedded in the ORF name.
looks like input config fiel is in fasta format.
doesn't look like input contig file is in gff format.  Will assume that contig name is embedded in the ORF name.
looks like input config fiel is in fasta format.
doesn't look like input contig file is in gff format.  Will assume that contig name is embedded in the ORF name.
looks like input config fiel is in fasta format.
doesn't look like input contig file is in gff format.  Will assume that contig name is embedded in the ORF name.
looks like input config fiel is in fasta format.
doesn't look like input contig file is in gff format.  Will assume that contig name is embedded in the ORF name.
looks like input config fiel is in fasta format.
doesn't look like input contig file is in gff format.  Will assume that contig name is embedded in the ORF name.
looks like input config fiel is in fasta format.
doesn't look like input cont

doesn't look like input contig file is in gff format.  Will assume that contig name is embedded in the ORF name.
looks like input config fiel is in fasta format.
doesn't look like input contig file is in gff format.  Will assume that contig name is embedded in the ORF name.
looks like input config fiel is in fasta format.
doesn't look like input contig file is in gff format.  Will assume that contig name is embedded in the ORF name.
looks like input config fiel is in fasta format.
doesn't look like input contig file is in gff format.  Will assume that contig name is embedded in the ORF name.
looks like input config fiel is in fasta format.
doesn't look like input contig file is in gff format.  Will assume that contig name is embedded in the ORF name.
looks like input config fiel is in fasta format.
doesn't look like input contig file is in gff format.  Will assume that contig name is embedded in the ORF name.
looks like input config fiel is in fasta format.
doesn't look like input cont

doesn't look like input contig file is in gff format.  Will assume that contig name is embedded in the ORF name.
looks like input config fiel is in fasta format.
doesn't look like input contig file is in gff format.  Will assume that contig name is embedded in the ORF name.
looks like input config fiel is in fasta format.
doesn't look like input contig file is in gff format.  Will assume that contig name is embedded in the ORF name.
looks like input config fiel is in fasta format.
doesn't look like input contig file is in gff format.  Will assume that contig name is embedded in the ORF name.
looks like input config fiel is in fasta format.
doesn't look like input contig file is in gff format.  Will assume that contig name is embedded in the ORF name.
looks like input config fiel is in fasta format.
doesn't look like input contig file is in gff format.  Will assume that contig name is embedded in the ORF name.
looks like input config fiel is in fasta format.
doesn't look like input cont

doesn't look like input contig file is in gff format.  Will assume that contig name is embedded in the ORF name.
looks like input config fiel is in fasta format.
doesn't look like input contig file is in gff format.  Will assume that contig name is embedded in the ORF name.
looks like input config fiel is in fasta format.
doesn't look like input contig file is in gff format.  Will assume that contig name is embedded in the ORF name.
looks like input config fiel is in fasta format.
doesn't look like input contig file is in gff format.  Will assume that contig name is embedded in the ORF name.
looks like input config fiel is in fasta format.
doesn't look like input contig file is in gff format.  Will assume that contig name is embedded in the ORF name.
looks like input config fiel is in fasta format.
doesn't look like input contig file is in gff format.  Will assume that contig name is embedded in the ORF name.
looks like input config fiel is in fasta format.
doesn't look like input cont

In [34]:
still_problematic = []
for sag in summary_generation_required:
    try:
        shutil.move("/mnt/scgc/simon/simonsproject/bats248_vs/diamond/pergenome/{}__proteins_mg_diamond_recruitment_tbl.csv".format(sag),
               "/mnt/scgc/simon/simonsproject/bats248_vs/diamond/pergenome/{}_proteins_mg_diamond_recruitment_tbl.csv".format(sag))
    except:
        still_problematic.append(sag)

In [35]:
still_problematic

['AG-904-P07',
 'AG-905-J06',
 'AG-909-J16',
 'AG-910-J09',
 'AG-910-P20',
 'AG-917-P16',
 'AG-918-A17',
 'AG-918-P23',
 'AG-894-J09']

In [37]:
read_count_dict = {'vir_reads':5922080, 'bac_reads':8279226}

missing = []
for sag in summary_generation_required:
    bac_tsv = "/mnt/scgc/simon/simonsproject/bats248_vs/diamond/pergenome/{}_proteins_vs_LineP-all.tsv.gz".format(sag)
    vir_tsv = "/mnt/scgc/simon/simonsproject/bats248_vs/diamond/pergenome/{}_proteins_vs_POV.tsv.gz".format(sag)
    plate = "-".join(sag.split("-")[:2])
    contig_file = "/mnt/scgc/simon/simonsproject/bats248_contigs/coassemblies/{plate}/{sag}_contigs.fasta".format(plate=plate, sag=sag)
    out_tbl = "/mnt/scgc/simon/simonsproject/bats248_vs/diamond/pergenome/{sag}_proteins_mg_diamond_recruitment_tbl.csv".format(sag=sag)
    if op.exists(out_tbl):
        continue
        
    exists=True
    for f in [bac_tsv, vir_tsv, contig_file]:

        if not op.exists(f):
            exists = False
            missing.append(sag)
            print(f)
            continue
       
    if exists == False:
        continue
            
    tbl = construct_recruit_tbl(vir_tsv, bac_tsv, read_count_dict, contig_file)      
    tbl.to_csv(out_tbl)

/mnt/scgc/simon/simonsproject/bats248_contigs/coassemblies/AG-918/AG-918-A17_contigs.fasta
/mnt/scgc/simon/simonsproject/bats248_contigs/coassemblies/AG-918/AG-918-P23_contigs.fasta
/mnt/scgc/simon/simonsproject/bats248_contigs/coassemblies/AG-894/AG-894-J09_contigs.fasta


In [39]:
merge_error

['AG-904-P07', 'AG-905-J06', 'AG-905-K20', 'AG-910-M04', 'AG-921-M10']

In [40]:
sag = merge_error[0]

In [45]:
prot_fasta = op.join(wd, 'prodigal', '{}_proteins.fasta'.format(sag))
recruit_csv = op.join(wd, 'diamond','pergenome','{}_proteins_mg_diamond_recruitment_tbl.csv'.format(sag))
out_csv = op.join(wd, 'summary', '{}_summary.csv'.format(sag))
phage_df = phage_contig_table(cmap, prot_fasta, phage_hits, outfile=None)
recruit_df = pd.read_csv(recruit_csv)

merged = recruit_df.merge(phage_df.reset_index(), on='contig', how='outer')
    


In [46]:
merged

,Unnamed: 0,contig,hit_mg-bac,hit_mg-vir,reads_mg-vir,reads_mg-bac,contig_length,fr_mg-bac,fr_mg-vir,ratio_virus_bacteria,p1,p2,total_orfs,viral_phage_gene_fraction,viral2_phage_gene_fraction
0,0.0,AG-904-P07_NODE_3,NaN,1.0,5922080.0,8279226.0,4517.0,NaN,0.000037,NaN,0.0,0.0,3,0.000000,0.0
1,NaN,AG-904-P07_NODE_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,7,0.000000,0.0
2,NaN,AG-904-P07_NODE_2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,2,0.000000,0.0
3,NaN,AG-904-P07_NODE_4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,3,0.333333,0.0


In [48]:
import pandas as pd
import glob
import os
import os.path as op
from collections import Counter

from bisect import bisect_left
import numpy as np
from sklearn import neighbors
import pandas as pd
from math import log




def virus_class(vsdf, test_cols = ['viral_phage_gene_fraction','ratio_virus_bacteria'], 
                training_file = '/mnt/scgc_nfs/opt/viruscope/virus-training.csv', 
                k = 27, score_col='score'):
    ''' function for determining virus class using KNeighborsClassifier
    Args:
        vsdf (pandas.DataFrame): viruscope dataframe having the the two calculated test columns and a contigs column included
        test_cols (list): list of names for test columns to be used for KNeighbors Classifier
        training_file (path): path to the training file for the knn algorithm
        k (int): number of nearest neighbors
        score_col(str): name of score column to use for training
    Returns:
        dataframe with columns: contig, virus_class, probability
    '''
    def _pval_by_count(clf, test_set, training_df, 
                  test_cols=['viral_phage_gene_fraction','ratio_virus_bacteria'],
                 score_col='score'):
        
        ''' 
        function for calculating probability that matches the previously used R function 
        
        Args:
            clf: KNeighborsClassifier object
            test_set: dataframe used to test knn algorithm on
            training_df: dataframe used to train knn
            test_cols: columns in above dataframes used to train the knn algorithm
            score_col: column in training algorithm that reports the score
        Returns: 
            list of pvalues in same order as rows in test_set dataframe
        '''
        
        ns = clf.kneighbors(test[test_cols], return_distance=False)
        pvals = []

        for l in ns:
            ndf = training_df.loc[l,:]
            pvals.append(Counter(ndf['score']).most_common()[0][1] / len(ndf))
        return pvals
    
    train_in = pd.read_table(training_file, sep = ',').dropna(axis = 0, how = 'any').reset_index()
    train_in.rename(columns={'ratio.virus.bacteria':'ratio_virus_bacteria'}, inplace=True)
    train_in[test_cols[0]] = np.log10(train_in[test_cols[0]] + 0.001)
    train_in[test_cols[1]] = np.log10(train_in[test_cols[1]] + 0.001)
    train_label = list(train_in.iloc[:,-1].values)
    in_train = train_in[test_cols]
    in_train.columns = test_cols
    
    # make the classifier and fit the training the data
    
    clf = neighbors.KNeighborsClassifier(k, weights='distance', algorithm='auto')
    clf.fit(in_train, train_label)
    
    test = vsdf[['contig'] + test_cols].dropna()
    if len(test) == 0:
        return pd.DataFrame(columns=['contig','virus_class','virus_prob'])
    test[test_cols[0]] = np.log10(test[test_cols[0]] + 0.001)
    test[test_cols[1]] = np.log10(test[test_cols[1]] + 0.001)
    try:
        test['virus_class'] = clf.predict(test[test_cols])
    except Exception as inst:
        print(test)
        raise inst
    probs = clf.predict_proba(test[test_cols])
    # built in probability function
    test['prob 0'] = [i[0] for i in probs]
    test['prob 1'] = [i[1] for i in probs]
    test['virus_prob_new'] = [l['prob 0'] if l['virus_class'] == 0 else l['prob 1'] for i, l in test.iterrows()]
    
    test['virus_prob'] = _pval_by_count(clf, test, train_in, test_cols, score_col)
    test['virus_prob_new'] = 
    
    return test[['contig','virus_class','virus_prob','virus_prob_new']]

In [49]:
virus_class(merged)

,contig,virus_class,virus_prob


In [52]:
result = merged.merge(virus_class(merged), on='contig', how='outer')

In [53]:
result

,Unnamed: 0,contig,hit_mg-bac,hit_mg-vir,reads_mg-vir,reads_mg-bac,contig_length,fr_mg-bac,fr_mg-vir,ratio_virus_bacteria,p1,p2,total_orfs,viral_phage_gene_fraction,viral2_phage_gene_fraction,virus_class,virus_prob
0,0.0,AG-904-P07_NODE_3,NaN,1.0,5922080.0,8279226.0,4517.0,NaN,0.000037,NaN,0.0,0.0,3,0.000000,0.0,NaN,NaN
1,NaN,AG-904-P07_NODE_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,7,0.000000,0.0,NaN,NaN
2,NaN,AG-904-P07_NODE_2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,2,0.000000,0.0,NaN,NaN
3,NaN,AG-904-P07_NODE_4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,3,0.333333,0.0,NaN,NaN
